# LC.980 | Unique Paths III | `Hard`

### BFS & Ordered Set
#### Divergent Thinking
1. We can go in any compass direction
2. We can't go to the same square twice.
3. We want all unique paths.
4. We must avoid obstacles.
5. The target destination is unknown.

#### How to solve for a single path?
1. BFS: we want to pick the neighbor that is NOT the destination if the neighbor is unvisited. we prioritize non-destination nodes.
    - Gives us every possible continuation from the previous value. Builds every path, Bottom-up
2. DFS: We must use backtracking. If we use backtracking, how do we know we've looked at all squares? We could pre-scan the grid for obstacles to know how many possible squares are legally visitable, THEN conduct a DFS....2*(m*n) 🫤

1. Serialize every path as we explore
2. Save each path in a collection
3. If we cannot continue further: & we know we haven't seen all squares, we remove this path from the collection of possibilities.

#### how do we serialize a path?
    - so that any cell in a path, knows the previous cells?
Define "we cannot continue further"
    1. (i, j).neighbor == wall * there's no legal neighbors
        1.1 - continue and forget about this path
return solution
    1. If the last node in the path is the destination, add it to a list of results.
    2. Once we finish travelling thru the grid, we count the paths with the longest chain of cells.

ideas:
    - to validate we've travelled the whole grid, we could keep track of the wall spots.
    - then when we calculate the longest path in the results list, we can subtract m*n - walls and verify the values are equal.
#### Mental Model
```
    t=0
    Q = [
        #1 (0, 0)(0, 1)
        #2 (0, 0)(1, 0)
        # 1.1 (0, 0)(0, 1)+(0, 2)
        # 1.2 (0, 0)(0, 1)+(1, 1)
        # 2.1 (0, 0)(1, 0)+(1,1)
        # 2.2 (0, 0)(1, 0)+(2,0)
    ]

    t=1
    (0, 0)(0, 1)+(0, 2)
    (0, 0)(0, 1)+(1, 1)
    (0, 0)(1, 0)+(1,1)
    (0, 0)(1, 0)+(2,0)
```

In [ ]:
from collections import defaultdict, UserDict, deque


class OrderedSet(UserDict):
    data_list = []

    def peek(self):
        return self.data_list[-1]

    def add(self, tuple):
        self[tuple] = len(self.data) + 1
        self.data_list.append(tuple)
        return self

    def create(self, oset):
        self.data = {**oset}
        self.data_list = list(self.data.keys())
        return self


class Solution2:
    def uniquePathsIII(self, grid) -> int:
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])
        self.walls = set()
        for i, row in enumerate(grid):
            for j, _ in enumerate(row):
                if self.grid[i][j] == 1:
                    paths = self.search((i, j))  # TC = O(rows*cols*p) | SC =
                    return self.calc_answer(paths)

    def search(self, start):
        paths = defaultdict(int)
        q = deque([OrderedSet().add(start)])
        while q:
            path = q.pop()  # O(1)
            i, j = path.peek()  # O(1)
            if self.grid[i][j] == 2:
                paths[len(path)] += 1
            else:
                for n in self.get_neighbors(path):  # O(4)
                    next_path = OrderedSet().create(path).add(n)  # O(p)
                    q.appendleft(next_path)
        return paths

    def calc_answer(self, paths):
        target_path_len = (self.rows * self.cols) - len(self.walls)  # 5
        count = paths.get(target_path_len, 0)
        return count

    def get_neighbors(self, path):
        i, j = path.peek()
        neighbors = []
        for dx, dy in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            row, col = i + dx, j + dy
            in_bounds = 0 <= row < self.rows and 0 <= col < self.cols
            if not in_bounds:
                continue
            if self.grid[row][col] == -1:
                self.walls.add((row, col))
            elif (row, col) not in path:
                neighbors.append((row, col))
        return neighbors


### LC Solution
1. initialize the conditions for backtracking, i.e. initial state and final state
2. count of paths as the final result
3. base case for the termination of backtracking
4. mark the square as visited. case: 0, 1, 2
5. we now have one less square to visit
6. The actual "Backtracking" step: unmark the square after the visit

In [ ]:
# LeetCode's solution w/ DFS & Backtracking
class Solution:
    def uniquePathsIII(self, grid) -> int:
        self.grid = grid
        self.path_count = 0
        self.rows, self.cols = len(grid), len(grid[0])
        # Note 1
        non_obstacles = 0
        start_row, start_col = 0, 0
        for i, row in enumerate(grid):
            for j, _ in enumerate(row):
                cell = grid[i][j]
                if cell >= 0:
                    non_obstacles += 1
                if cell == 1:
                    start_row, start_col = i, j
        self.dfs_backtrack(start_row, start_col, non_obstacles)
        return self.path_count

    def dfs_backtrack(self, row, col, remain):
        if self.grid[row][col] == 2 and remain == 1:  # Note 3
            self.path_count += 1
            return

        temp = self.grid[row][col]  # Note 4
        self.grid[row][col] = -4
        remain -= 1   # Note 5

        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            i, j = row + dx, col + dy
            is_in_bounds = (0 <= i < self.rows and 0 <= j < self.cols)
            if is_in_bounds and not self.grid[i][j] < 0:
                self.dfs_backtrack(i, j, remain)
        self.grid[row][col] = temp  # Note 6


Solution().uniquePathsIII([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 2, -1]])
